In [22]:
import pandas as pd
from scipy.stats import ttest_ind
df = pd.read_csv('my_final_data.csv', sep = ',')

In [23]:
# Устанавливаем опции для отображения всех столбцов
pd.set_option('display.max_columns', None)       # Нет ограничения на количество отображаемых столбцов

df.head(1)

,Unnamed: 0,Game_ID,Rated,Variant,Speed,Created_At,Last_Move_At,Game_Status,Source,Winner,Opening_Name,Opening_ECO,Opening_PLY,White_Player,White_Title,White_Rating,White_Inaccuracy,White_Mistake,White_Blunder,White_ACPL,Black_Player,Black_Title,Black_Rating,Black_Inaccuracy,Black_Mistake,Black_Blunder,Black_ACPL,Clocks,Analysis,Total_Time_Init,Total_Time_Increment,Moves,Created_Year,Created_Month,weekday,Hour,Last_Move_Year,Last_Move_Month,Last_Move_Hour,White_ACPL_log,Black_ACPL_log,White_Clocks,White_Clocks_Diffs,White_Q50,White_Q75,White_Q90,Log_White_Q50,Log_White_Q75,Log_White_Q90,Black_Clocks,Black_Clocks_Diffs,Black_Q50,Black_Q75,Black_Q90,Log_Black_Q50,Log_Black_Q75,Log_Black_Q90,eval,avg_eval,eval_count,Total_Mistakes,Total_Time_Increment_Group,Is_Berserk_White,Is_Berserk_Black,Game_Duration,Log_Game_Duration,Num_Moves
0,0,PsevV6VH,True,standard,classical,2024-12-15 11:09:25,2024-12-15 11:55:14,resign,pool,white,English Opening: Symmetrical Variation,A30,3.0,orenlivne,NT,2334.0,1,1,1,31,oyto_971179789,NT,2272.0,5,1,1,47,"[180003, 180003, 178499, 178019, 178339, 17757...","[{'eval': 14}, {'eval': 25}, {'eval': 26}, {'e...",1800,0,Nf3 c5 c4 Nc6 Nc3 g6 e3 Bg7 d4 cxd4 exd4 d6 h3...,2024,12,Sunday,11,2024,12,11,3.465736,3.871201,"[180003, 178499, 178339, 178131, 177939, 17793...","[15.04, 1.6, 2.08, 1.92, 0.0, 77.2, 55.68, 56....",15.64,54.28,101.22,2.811809,4.012411,4.627127,"[180003, 178019, 177571, 173699, 173203, 17218...","[19.84, 4.48, 38.72, 4.96, 10.16, 183.36, 26.8...",27.76,45.68,62.47,3.358986,3.843316,4.150567,"[14, 25, 26, 23, 16, 39, 32, 44, 51, 89, 90, 8...",186.257576,66,2,0 sec,False,False,2749.0,7.919356,34


Формулируем гипотезы:

Нулевая гипотеза (H0): Игра с сильнейшим противником не влияет на общее количество ошибок и неточностей.

Альтернативная гипотеза (HA): Игра с сильнейшим противником увеличивает общее количество ошибок и неточностей.

In [24]:
# Этап 1: Создание общего числа ошибок для обоих игроков
df['Total_Errors_WHITE'] = df['White_Inaccuracy'] + df['White_Mistake'] + df['White_Blunder']
df['Total_Errors_BLACK'] = df['Black_Inaccuracy'] + df['Black_Mistake'] + df['Black_Blunder']

# Этап 2: Определение сильных соперников (предположим порог различия в рейтинге 325)
rating_difference_threshold = 325

# Вычисляем разность рейтинга (белый игрок минус черный игрок)
df['Rating_Difference'] = abs(df['White_Rating'] - df['Black_Rating'])

# Группируем матчи на основании разницы в рейтинге
strong_opponent_games = df[df['Rating_Difference'] >= rating_difference_threshold]
weak_or_average_opponent_games = df[df['Rating_Difference'] < rating_difference_threshold]

# Этап 3: Извлекаем суммарные ошибки для обеих групп (отдельно для черных и белых)
total_errors_strong_white = strong_opponent_games['Total_Errors_WHITE'].values
total_errors_strong_black = strong_opponent_games['Total_Errors_BLACK'].values

total_errors_weak_white = weak_or_average_opponent_games['Total_Errors_WHITE'].values
total_errors_weak_black = weak_or_average_opponent_games['Total_Errors_BLACK'].values

# Этап 4: Применение t-теста для оценки различий между группами
# Проверка изменения качества игры белого игрока
statistic_white, p_value_white = ttest_ind(total_errors_strong_white, total_errors_weak_white)
print(f"Тестовое значение (белый игрок): {statistic_white}, P-value: {p_value_white}")

# Анализ результатов
if p_value_white < 0.05:
    print("Гипотеза отклоняется: сила противника существенно влияет на количество ошибок.")
else:
    print("Гипотеза принимается: нет доказательств существенного влияния силы противника на количество ошибок.")
    
# Проверка изменения качества игры черного игрока
statistic_black, p_value_black = ttest_ind(total_errors_strong_black, total_errors_weak_black)
print(f"Тестовое значение (черный игрок): {statistic_black}, P-value: {p_value_black}")

# Анализ результатов
if p_value_black < 0.05:
    print("Гипотеза отклоняется: сила противника существенно влияет на количество ошибок.")
else:
    print("Гипотеза принимается: нет доказательств существенного влияния силы противника на количество ошибок.")

Тестовое значение (белый игрок): -2.1614931567646543, P-value: 0.030659367732580007
Гипотеза отклоняется: сила противника существенно влияет на количество ошибок.
Тестовое значение (черный игрок): -5.860439247408993, P-value: 4.629258892176753e-09
Гипотеза отклоняется: сила противника существенно влияет на количество ошибок.


ВЫВОД:

Используем подходящий статистический критерий для сравнения средних значений двух групп (игр с сильными игроками и игр со средними/более слабыми). Наиболее подходящим методом здесь является t-тест, поскольку мы сравниваем средние значения между двумя независимыми выборками.

По нашему тесту видно, что существует прямая зависимость между силой игрока и количеством допускаемых ошибок при игре с ним. НО эта зависимость есть, если разница в рейтинге более 325. Этот порог выявили экспериментальным путем подбора разницы рейтингов, так как если рейтинг ниже 325, то у белых игроков такой зависимости уже не наблюдается.